In [0]:
spark.conf.set('spark.databricks.io.cache.enabled',False)

In [0]:
from pyspark.sql.functions import *

transactions_df = (spark
                        .range(0, 1500000000, 1, 32)
                        .select(
                            'id',
                            round(rand() * 10000, 2).alias('amount'),
                            (col('id') % 10).alias('country_id'),
                            (col('id') % 100).alias('store_id')
)
)


In [0]:
transactions_df.write.mode("overwrite").saveAsTable("cgi_dev.swetha.transactions")

In [0]:
%sql
describe detail cgi_dev.swetha.transactions

In [0]:
%sql
describe extended cgi_dev.swetha.transactions

In [0]:
stores_df = (spark
                        .range(0, 99)
                        .select(
                            'id',
                            round(rand() * 100,0).alias('employees'),
                            (col('id') % 10).alias('country_id'),
                            expr('uuid()').alias('name')
)
)

stores_df.write.mode("overwrite").saveAsTable("cgi_dev.swetha.stores")

In [0]:
countries =[(0, "Italy"),
            (1, "Canada"),
            (2, "Mexico"),
            (3, "China"),
            (4, "Germany"),
            (5, "UK"),
            (6, "Japan"),
            (7, "Korea"),
            (8, "Australia"),
            (9, "France"),
            (10, "Spain"),
            (11, "USA")
            ]

columns = ["id", "name"]

countries_df = spark.createDataFrame(data = countries, schema = columns)

countries_df.write.mode("overwrite").saveAsTable("cgi_dev.swetha.countries")

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.databricks.adaptive.autoBroadcast.JoinThreshold", -1)

In [0]:
%sql
 SELECT
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        cgi_dev.swetha.transactions
    LEFT JOIN
        cgi_dev.swetha.stores
        ON
            transactions.store_id = stores.id
        LEFT JOIN
            cgi_dev.swetha.countries
            ON
                transactions.country_id = countries.id

In [0]:
spark.conf.unset("spark.sql.autoBroadcastJoinThreshold")
spark.conf.unset("spark.databricks.adaptive.auteBroadcastJoinThreshold")

In [0]:
%sql
 SELECT
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        cgi_dev.swetha.transactions
    LEFT JOIN
         cgi_dev.swetha.stores
        ON
            transactions.store_id = stores.id
        LEFT JOIN
            cgi_dev.swetha.countries
            ON
                transactions.country_id = countries.id